In [ ]:
!pip install ekphrasis

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 184kB 8.3MB/s 
     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=7559d902dd0ecc4b606504f7e57c58dd699879e3c81c09ebe444e011e5e5ea3d
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=d696d395b454435f1ab75fb6433ba344722131a189d5dac6d6f3a5ad37f7cdb1
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ekphrasis ftfy


In [ ]:
!pip install transformers==4.2.1

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 870kB 25.2MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9917ae9a290c381c08111ae5425ed1601005d4b2e6d08f66b195a4faa51cb330
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
pip install tf-models-official

     |████████████████████████████████| 1.1MB 6.6MB/s 
     |████████████████████████████████| 645kB 19.4MB/s 
     |████████████████████████████████| 1.2MB 25.5MB/s 
     |████████████████████████████████| 706kB 35.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 358kB 41.9MB/s 
     |████████████████████████████████| 174kB 35.6MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 37.6MB 1.2MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=0357724159bb24b2e7052632d39049075c61e0fb9eef197b1ed7851dc23ed2b7
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=3e9eba69514d789543f01cd4c05a5d13c6891802186953d5b19a09f7713ce8f0
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf487013

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter
from official import nlp
import official.nlp.optimization

import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.74.29.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.29.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... 

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/data'
!ls

/content/drive/MyDrive/data
albert-1-b.csv	   albert-2.csv       answer-1-b.csv	    task2.csv
albert-1-b.gsheet  albert-2.gsheet    dev		    test
albert-1-c.csv	   answer-1-a.csv     Dexter_answer.gsheet  train
albert-1-c.gsheet  answer-1-a.gsheet  slang.txt


In [ ]:
df_train = pd.read_csv('./train/train.csv', encoding='utf-8')
df_train['humor_rating'] = df_train['humor_rating'].fillna(0)
df_train['humor_controversy'] = df_train['humor_controversy'].fillna(2)
text_train = df_train["text"]
df_train.head(10)

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10
5,6,"'Trabajo,' the Spanish word for work, comes fr...",0,0.00,2.0,0.00
6,7,I enrolled on some skill training and extra cu...,0,0.00,2.0,0.10
7,8,ME: I'm such an original. Truly one of a kind....,1,1.79,1.0,0.00
8,9,Men who ejaculated 21 times or more a month ha...,0,0.00,2.0,0.05
9,10,I got REALLY angry today and it wasn't about n...,0,0.00,2.0,0.15


In [ ]:
df_val = pd.read_csv('./dev/dev.csv', encoding='utf-8')
df_val['humor_rating'] = df_val['humor_rating'].fillna(0)
df_val['humor_controversy'] = df_val['humor_controversy'].fillna(2)
text_val = df_val["text"]
df_val.head(10)

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,8001,What's the difference between a Bernie Sanders...,1,2.45,0.0,1.70
1,8002,"Vodka, whisky, tequila. I'm calling the shots.",1,2.00,0.0,0.00
2,8003,French people don't masturbate They Jacque off,1,2.95,0.0,1.15
3,8004,A lot of Suicide bombers are Muslims - I don't...,1,1.38,0.0,3.75
4,8005,What happens when you fingerbang a gypsy on he...,1,2.70,1.0,2.25
5,8006,Who's the biggest slut ever? Mrs. Pacman--for ...,1,3.05,1.0,2.10
6,8007,Caught a tall black man looking through my win...,1,1.89,0.0,1.80
7,8008,"A6: It CAN be, but it doesn't have to be. It d...",0,0.00,2.0,0.00
8,8009,Parenting Takes Mom and Dad.. So why do we hum...,0,0.00,2.0,1.00
9,8010,Ok folks-I'm gonna try this live tweeting thin...,0,0.00,2.0,0.00


In [ ]:
df_test = pd.read_csv('./test/public_test.csv', encoding='utf-8')
text_test = df_test["text"]
df_test.head(10)

,id,text
0,9001,Finding out your ex got fat is like finding 20...
1,9002,"For Brockmann, stereotypes imperil national se..."
2,9003,A girl runs up to her mother with a pile of cr...
3,9004,gotta wonder if baseball still would've been c...
4,9005,When you're dreading getting in the shower cuz...
5,9006,Parenthood feels like you're on a magical unic...
6,9007,It's Friday night and I'm out of control! Gett...
7,9008,me: I didn't know this was a hibachi restauran...
8,9009,Maybe we need to turn the United States off an...
9,9010,I asked my North Korean friend how it was ther...


In [ ]:
print(len(text_train))
print(len(text_val))
print(len(text_test))

8000
1000
1000


In [ ]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [ ]:
# Functions for chat word conversion
f = open("./slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("-")[0]
        cw_expanded = line.split("-")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
# Chat word conversion
# Training set
text_train = text_train.apply(lambda text: chat_words_conversion(text))
print_text(text_train,0,10)

print("----------------------------------------------------------------")

# Validation set
text_val = text_val.apply(lambda text: chat_words_conversion(text))
print_text(text_val,0,10)
print("----------------------------------------------------------------")
# Test set
text_test = text_test.apply(lambda text: chat_words_conversion(text))
# print_text(text_test,0,10)

TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit...

A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step

How many men does it take to open a can of beer? None. It should be open by the time she brings it to the couch.

Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!

Roses are dead. Love is fake. Weddings are basically funerals with cake.

'Trabajo,' the Spanish word for work, comes from the Latin term 'trepaliare,' meaning torture.

I enrolled on some skill training and extra curricula activities that added shape to my weekends and after-work schedule. The more days passed, the more exposed I was to how blindly in love I was with an obsession. It's been months now and we are just cool as men.

ME

In [ ]:
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

In [ ]:
# Training set
text_train = text_train.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_val = text_val.apply(lambda text: ekphrasis_pipe(text))
print("Validation set completed.......")
#Test set
text_test = text_test.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

Training set completed.......
Validation set completed.......
Test set completed.......


In [ ]:
# Finding length of longest array
maxLen = len(max(text_train,key = lambda text: len(text.split(" "))).split(" "))

In [ ]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_train:
    sentence_lengths.append(u(x))
print(len(sentence_lengths))

8000


In [ ]:
is_humor = df_train["is_humor"]
humor_rating = df_train["humor_rating"]
humor_controversy = df_train["humor_controversy"].astype(int)
offense_rating = df_train["offense_rating"]
print(Counter(is_humor))
print(Counter(humor_controversy))

Counter({1: 4932, 0: 3068})
Counter({2: 3068, 0: 2467, 1: 2465})


In [ ]:
is_humor_val = df_val["is_humor"]
humor_rating_val = df_val["humor_rating"]
humor_controversy_val = df_val["humor_controversy"].astype(int)
offense_rating_val = df_val["offense_rating"]
print(Counter(is_humor_val))
print(Counter(humor_controversy_val))

Counter({1: 632, 0: 368})
Counter({2: 368, 0: 324, 1: 308})


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
# Define tokenizer as per requirement
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
text_train = list(text_train)
text_val = list(text_val)
text_test = list(text_test)
train_encodings = tokenizer(text_train, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(text_val, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(text_test, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
print(np.shape(train_encodings["input_ids"]))
print(np.shape(val_encodings["input_ids"]))
print(np.shape(test_encodings["input_ids"]))
print(train_encodings["input_ids"][0])
print("***************************************************************************")
print(val_encodings["input_ids"][0])

(8000, 150)
(1000, 150)
(1000, 150)
tf.Tensor(
[  101  1026  2035 17695  2015  1028  5298  1026  1013  2035 17695  2015
  1028  1024  2057  2024  1996  2190  2110  1012  6343  2130  3310  2485
  1012  1008  5408  2791  4402  7365  2046  1996  2282  1008  1026  2035
 17695  2015  1028  5298  1026  1013  2035 17695  2015  1028  1024  2821
  4485  1012  1026  5567  1028   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int32)
***********

In [ ]:
def task_1(input_shape):
    # Import model as required
    model = TFBertModel.from_pretrained('bert-base-uncased')
    layer = model.layers[0]
    #Model
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')

    outputs = layer([inputs, input_masks])
    output = outputs[0]
    pooled_output = output[:, 0, :]
    # Humour classification
    is_humor = layers.Dropout(0.3)(pooled_output)
    # is_humor = layers.Dense(128, activation="gelu")(is_humor)
    is_humor = layers.Dense(1, activation="sigmoid")(is_humor)

    model = keras.Model(inputs=[inputs,input_masks], outputs=is_humor, name='Task_1_a')
    
    return model

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
    model = task_1((150,))
    optimizer = keras.optimizers.Adam(learning_rate=2e-5)
    loss_fun = [
          tf.keras.losses.BinaryCrossentropy(from_logits=False)
    ]
    metric = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model.summary()

Model: "Task_1_a"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/task-1-a-model-name.{epoch:03d}.h5',
                                 verbose = 0,
                                 save_weights_only=True,
                                 epoch=4)
count_humor = Counter(is_humor)
zero = count_humor[0]
one = count_humor[1]
total = zero + one
print("Not Humorous: ",zero)
print("Humorous: ",one)
print("Total: ",total)

Not Humorous:  3068
Humorous:  4932
Total:  8000


In [ ]:
class_weight_ = {}
maxi = max(zero, one)
weight_for_0 = (maxi / (maxi + zero))
weight_for_1 = (maxi / (maxi + one))

class_weight_motivation = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.62
Weight for class 1: 0.50


In [ ]:
import time
start_time = time.time()
history_task_1_a = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = is_humor,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]], is_humor_val),
    callbacks = [checkpoint],
    batch_size=16,
    class_weight=class_weight_,
    shuffle=True,
    epochs=4)
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/4


500/500 [==============================] - 101s 88ms/step - loss: 0.3201 - binary_accuracy: 0.8514 - precision: 0.8676 - recall: 0.8965 - val_loss: 0.2518 - val_binary_accuracy: 0.9020 - val_precision: 0.9045 - val_recall: 0.9446
Epoch 2/4
500/500 [==============================] - 39s 78ms/step - loss: 0.0819 - binary_accuracy: 0.9700 - precision: 0.9768 - recall: 0.9748 - val_loss: 0.2727 - val_binary_accuracy: 0.9020 - val_precision: 0.9279 - val_recall: 0.9161
Epoch 3/4
500/500 [==============================] - 39s 79ms/step - loss: 0.0203 - binary_accuracy: 0.9942 - precision: 0.9955 - recall: 0.9952 - val_loss: 0.4608 - val_binary_accuracy: 0.8860 - val_precision: 0.8743 - val_recall: 0.9573
Epoch 4/4
500/500 [==============================] - 39s 78ms/step - loss: 0.0116 - binary_accuracy: 0.9960 - precision: 0.9966 - recall: 0.9969 - val_loss: 0.4859 - val_binary_accuracy: 0.8920 - val_precision: 0.8946 - val_recall: 0.9399
--- 228.70286560058594 seconds ---


In [ ]:
import time
start_time = time.time()
val_answer = model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])
val_answer = np.round(val_answer)
val_answer = np.squeeze(val_answer, axis=-1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.188027620315552 seconds ---


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(is_humor_val, val_answer, digits=4))

              precision    recall  f1-score   support

           0     0.8869    0.8098    0.8466       368
           1     0.8946    0.9399    0.9167       632

    accuracy                         0.8920      1000
   macro avg     0.8907    0.8748    0.8816      1000
weighted avg     0.8918    0.8920    0.8909      1000



In [ ]:
test_answer = model.predict([test_encodings["input_ids"],test_encodings["attention_mask"]])
test_answer = np.round(test_answer)
test_answer = np.squeeze(test_answer, axis=-1)
np.shape(test_answer)

(1000,)

In [ ]:
test_id = df_test["id"]

In [ ]:
test_dict = {
    "id" : test_id,
    "is_humor" : test_answer
}

df_test = pd.DataFrame(test_dict)
df_test.head()
df_test.to_csv('answer-1-a.csv', index=False)

In [ ]:
df_test

,id,is_humor
0,9001,1.0
1,9002,0.0
2,9003,1.0
3,9004,1.0
4,9005,1.0
...,...,...
995,9996,1.0
996,9997,1.0
997,9998,1.0
998,9999,0.0
